### Import Necessary Libaries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

!conda install -c conda-forge geopy --yes # Uncomment out if you need to install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values


# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes # Uncomment out you need to install folium
import folium # plotting library

!pip install xlrd


print('Folium installed')
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 

After importing/installing all the necessary packages, we will read the excel spreadsheet. The neighborhood and location columns are from wikipedia. I added the city, state, and combo column to allow geolocator to work in the following code.

In [2]:
# Reading table into dataframe
bmore_df = pd.read_excel('Baltimore_Neigh_List.xlsx')
print(bmore_df.shape)
bmore_df

(221, 5)


,Neighborhood,City,Location,State,Combo
0,Abell,Baltimore,North,MD,"Abell, Baltimore, MD"
1,Allendale,Baltimore,West,MD,"Allendale, Baltimore, MD"
2,Arcadia,Baltimore,Northeast,MD,"Arcadia, Baltimore, MD"
3,Arlington,Baltimore,Northwest,MD,"Arlington, Baltimore, MD"
4,Armistead Gardens,Baltimore,Northeast,MD,"Armistead Gardens, Baltimore, MD"
...,...,...,...,...,...
216,Woodring,Baltimore,Northeast,MD,"Woodring, Baltimore, MD"
217,Wyman Park,Baltimore,North,MD,"Wyman Park, Baltimore, MD"
218,Wyndhurst,Baltimore,North,MD,"Wyndhurst, Baltimore, MD"
219,Yale Heights,Baltimore,Southwest,MD,"Yale Heights, Baltimore, MD"


Here we are reading the data in the combo column using a for-loop

In [3]:
# allows to read the specific column of the spreadsheet
def readcolumn(filename,column):
    #select sheet name and selct column as index,index_col=0
    df = pd.read_excel(filename)
    headername = list(df)
    print(headername)
    column_data =df[list(df)[column]].tolist()
    return  column_data

neigh_list = readcolumn('Baltimore_Neigh_List.xlsx', 4)

['Neighborhood', 'City', 'Location', 'State', 'Combo']


Since the city names are not unique, I had to add in 'Baltimore' to the code so the geolocator would pull the correct lat/long coordinates.

In [13]:
# Prints out the neighborhood list with the latitude and longitude
df = []
for neigh in neigh_list:
    try:
        geolocator = Nominatim(user_agent="bmore_explorer")
        location = geolocator.geocode(neigh)
        latitude = location.latitude
        longitude = location.longitude
        print(neigh, latitude, longitude)
        df.append({'Location':location,
                  'Latitude': latitude,
                  'Longitude': longitude})
    except Exception as e:
        print('Error, skipping address...', e)

Abell, Baltimore, MD 39.3261388 -76.611737
Allendale, Baltimore, MD 39.2916866 -76.6769424
Arcadia, Baltimore, MD 39.3340564 -76.5664996
Arlington, Baltimore, MD 39.3486919 -76.6826661
Armistead Gardens, Baltimore, MD 39.3066459 -76.5562793455641
Arundel Cove, Baltimore, MD 39.1985156 -76.5673626074159
Ashburton, Baltimore, MD 39.3279621 -76.6710811
Baltimore Highlands, Baltimore, MD 39.2331629 -76.636634
Barclay, Baltimore, MD 39.3149545 -76.6122023
Barre Circle, Baltimore, MD 39.2848699 -76.6276523
Bayview, Baltimore, MD 39.2894325 -76.5416232
Beechfield, Baltimore, MD 39.2752308 -76.6975606
Belair-Edison, Baltimore, MD 39.3215607 -76.5678685
Belair-Parkside, Baltimore, MD 39.327438 -76.566113
Berea, Baltimore, MD 39.3092351 -76.5779746
Beverly Hills, Baltimore, MD 39.3402861 -76.5694739
Biddle Street, Baltimore, MD 39.3093234 -76.5260695
Bolton Hill, Baltimore, MD 39.3084715 -76.6259787
Brewer's Hill, Baltimore, MD 39.2838792 -76.5645376
Broadway East, Baltimore, MD 39.308818 -76.58

In [14]:
# Convert list to dataframe
df = pd.DataFrame.from_dict (df)
df

,Location,Latitude,Longitude
0,"(Abell, Baltimore, Maryland, 21218, USA, (39.3...",39.326139,-76.611737
1,"(Allendale, Baltimore, Maryland, 21229, USA, (...",39.291687,-76.676942
2,"(Arcadia, Baltimore, Maryland, 21082:21234, US...",39.334056,-76.566500
3,"(Arlington, Baltimore, Maryland, 21215, USA, (...",39.348692,-76.682666
4,"(Armistead Gardens, Baltimore, Maryland, USA, ...",39.306646,-76.556279
...,...,...,...
216,"(Woodring, Baltimore, Maryland, 21256, USA, (3...",39.363819,-76.544286
217,"(Wyman Park, Baltimore, Maryland, 21211, USA, ...",39.333540,-76.626371
218,"(Wyndhurst, Baltimore, Maryland, 21210, USA, (...",39.355871,-76.631357
219,"(Yale Heights, Baltimore, Maryland, 21229, USA...",39.275706,-76.689895


Let's merge the dataframes!

In [15]:
# Merging the two dataframes
df_merged = df.merge(bmore_df, how='outer', left_index=True, right_index=True)
df_merged

,Location_x,Latitude,Longitude,Neighborhood,City,Location_y,State,Combo
0,"(Abell, Baltimore, Maryland, 21218, USA, (39.3...",39.326139,-76.611737,Abell,Baltimore,North,MD,"Abell, Baltimore, MD"
1,"(Allendale, Baltimore, Maryland, 21229, USA, (...",39.291687,-76.676942,Allendale,Baltimore,West,MD,"Allendale, Baltimore, MD"
2,"(Arcadia, Baltimore, Maryland, 21082:21234, US...",39.334056,-76.566500,Arcadia,Baltimore,Northeast,MD,"Arcadia, Baltimore, MD"
3,"(Arlington, Baltimore, Maryland, 21215, USA, (...",39.348692,-76.682666,Arlington,Baltimore,Northwest,MD,"Arlington, Baltimore, MD"
4,"(Armistead Gardens, Baltimore, Maryland, USA, ...",39.306646,-76.556279,Armistead Gardens,Baltimore,Northeast,MD,"Armistead Gardens, Baltimore, MD"
...,...,...,...,...,...,...,...,...
216,"(Woodring, Baltimore, Maryland, 21256, USA, (3...",39.363819,-76.544286,Woodring,Baltimore,Northeast,MD,"Woodring, Baltimore, MD"
217,"(Wyman Park, Baltimore, Maryland, 21211, USA, ...",39.333540,-76.626371,Wyman Park,Baltimore,North,MD,"Wyman Park, Baltimore, MD"
218,"(Wyndhurst, Baltimore, Maryland, 21210, USA, (...",39.355871,-76.631357,Wyndhurst,Baltimore,North,MD,"Wyndhurst, Baltimore, MD"
219,"(Yale Heights, Baltimore, Maryland, 21229, USA...",39.275706,-76.689895,Yale Heights,Baltimore,Southwest,MD,"Yale Heights, Baltimore, MD"


In [16]:
# Removing the whitespace and renaming
df_merged.columns = ['location', 'Latitude', 'Longitude', 'Neighborhood', 'City', 'Location', 'State', 'Combo', ]
print(df_merged.columns.tolist())

['location', 'Latitude', 'Longitude', 'Neighborhood', 'City', 'Location', 'State', 'Combo']


I tried to drop the columns I did not need, but was running into an error, so I just created a new dataframe to bypass the error.

In [17]:
# Creating a new dataframe with the necessary columns
df_final = df_merged[['Neighborhood', 'Location', 'Latitude', 'Longitude']]
df_final

,Neighborhood,Location,Latitude,Longitude
0,Abell,North,39.326139,-76.611737
1,Allendale,West,39.291687,-76.676942
2,Arcadia,Northeast,39.334056,-76.566500
3,Arlington,Northwest,39.348692,-76.682666
4,Armistead Gardens,Northeast,39.306646,-76.556279
...,...,...,...,...
216,Woodring,Northeast,39.363819,-76.544286
217,Wyman Park,North,39.333540,-76.626371
218,Wyndhurst,North,39.355871,-76.631357
219,Yale Heights,Southwest,39.275706,-76.689895


In [18]:
# Ensure the types are correct for mapping
print(df_final.dtypes)

Neighborhood     object
Location         object
Latitude        float64
Longitude       float64
dtype: object


We will now see if any data was not merged correctly or if any is not present

In [19]:
# Calculate how many rows are missing
missing_data = df_final.isnull()

for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")  

Neighborhood
False    221
Name: Neighborhood, dtype: int64

Location
False    221
Name: Location, dtype: int64

Latitude
False    221
Name: Latitude, dtype: int64

Longitude
False    221
Name: Longitude, dtype: int64



It looks like all of the data was transferred correctly. 221 out of 221 rows!

# -------------------------------------------------------------------------------------

## Explore and cluster the neighborhoods in Baltimore

In [20]:
# create map of Baltimore using latitude and longitude values from above
map_baltimore = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map from 'can_loc' df
for lat, lng, location, neighborhood in zip(df_final['Latitude'], df_final['Longitude'], df_final['Location'], df_final['Neighborhood']):
    label = '{}, {}'.format(location, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_baltimore)  
    
map_baltimore

In [21]:
# View Neighborhood's total by cardinal direction
pd.value_counts(df_final['Location'].values, sort=False)

Northeast    31
Southeast    23
West         18
Southwest    26
Central      18
Northwest    27
North        44
East         16
South        18
dtype: int64

### Define credentials

In [22]:
CLIENT_ID = 'ENTER YOUR CLIENT_ID HERE' # your Foursquare ID
CLIENT_SECRET = 'ENTER YOUR CLIENT_SECRET HERE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + 'ENTERED')
print('CLIENT_SECRET:' + 'ENTERED')

Your credentails:
CLIENT_ID: ENTERED
CLIENT_SECRET:ENTERED


Finding the top 100 venues around Baltimore

In [23]:
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
print('URL entered with credentials')

URL entered with credentials


In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d7132f666fc65002c928719'},
 'response': {'venues': [{'id': '4e5235e9b61c097de677359c',
    'name': 'MTA #48  York and Coldspring',
    'location': {'lat': 39.34611835982064,
     'lng': -76.60937547761814,
     'labeledLatLngs': [{'label': 'display',
       'lat': 39.34611835982064,
       'lng': -76.60937547761814}],
     'distance': 21,
     'cc': 'US',
     'state': 'Maryland',
     'country': 'United States',
     'formattedAddress': ['Maryland', 'United States']},
    'categories': [{'id': '52f2ab2ebcbc57f1066b8b4f',
      'name': 'Bus Stop',
      'pluralName': 'Bus Stops',
      'shortName': 'Bus Stop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/busstation_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1567699702',
    'hasPerk': False},
   {'id': '4e7cc57b483b1e7b5ae348ef',
    'name': 'Chicken Run',
    'location': {'address': '4711 York Rd',
     'lat': 39.346191162312884,
     'lng

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Location', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Since we want to find the popular venues by location, we are going to first analyze the data through geographical location and will later analyze each neighborhood.

In [26]:
baltimore_venues = getNearbyVenues(names=df_final['Location'],
                                   latitudes=df_final['Latitude'],
                                   longitudes=df_final['Longitude']
                                  )

North
West
Northeast
Northwest
Northeast
South
Northwest
Southeast
East
Southwest
Southeast
Southwest
Northeast
Northeast
East
Northeast
East
Central
Southeast
East
South
South
Southeast
Northwest
North
Southeast
Southwest
Central
North
Northeast
Northeast
Northwest
Central
North
North
South
Northwest
Northeast
North
West
Northwest
South
North
East
Southwest
Northwest
Northwest
Central
East
Northwest
East
West
Southeast
Southwest
West
Northeast
East
North
Southwest
Northwest
South
Southeast
Southeast
Northwest
Northeast
West
Southwest
Northeast
East
Northwest
North
Northeast
North
Southeast
Southeast
East
North
Northwest
North
Northwest
North
Northwest
Southeast
Southeast
South
West
North
South
West
Southeast
Northeast
Southwest
North
Northeast
Southwest
North
Northwest
Northeast
Central
Southwest
East
Southeast
Southeast
North
North
North
Southeast
North
North
South
Northwest
Northeast
North
West
Central
Northwest
Southeast
South
North
Northwest
Central
Central
South
Northeast
East
No

### Examine the new dataframe

Now lets view the new dataframe and shape

In [27]:
print(baltimore_venues.shape)
baltimore_venues.head()

(3879, 7)


,Location,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North,39.326139,-76.611737,32nd Street Farmer's Market,39.327362,-76.610851,Farmers Market
1,North,39.326139,-76.611737,Pete's Grille,39.327064,-76.609593,Breakfast Spot
2,North,39.326139,-76.611737,The Book Thing,39.325253,-76.610272,Bookstore
3,North,39.326139,-76.611737,Normal's Books & Records,39.326012,-76.609903,Record Shop
4,North,39.326139,-76.611737,Peabody Heights Brewery,39.324744,-76.610819,Brewery


In [28]:
# one hot encoding
baltimore_onehot = pd.get_dummies(baltimore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
baltimore_onehot['Location'] = baltimore_venues['Location']

In [29]:
# move neighborhood column to the first column
neighborhood = baltimore_onehot['Location']
baltimore_onehot.drop(labels=['Location'], axis=1,inplace = True)
baltimore_onehot.insert(0, 'Location', neighborhood)
baltimore_onehot

,Location,ATM,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,...,Vietnamese Restaurant,Volleyball Court,Waterfall,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3874,Northeast,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3875,Northeast,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3876,Northeast,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3877,Northeast,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
baltimore_onehot.shape

(3879, 291)

Adding the venues expanded the dataframe from (3872, 7) to (3872, 291)

We will now group the neighborhoods by taking the mean of the frequency of occurrence of each category

In [31]:
baltimore_grouped = baltimore_onehot.groupby('Location').mean().reset_index()
baltimore_grouped

,Location,ATM,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,...,Vietnamese Restaurant,Volleyball Court,Waterfall,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central,0.000000,0.000000,0.008000,0.002667,0.000000,0.029333,0.00000,0.010667,0.000000,...,0.008000,0.000000,0.000000,0.000000,0.000000,0.008000,0.000000,0.000000,0.000000,0.004000
1,East,0.000000,0.000000,0.000000,0.000000,0.000000,0.051724,0.00000,0.000000,0.000000,...,0.005747,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.005747
2,North,0.005176,0.001035,0.001035,0.000000,0.000000,0.033126,0.00207,0.000000,0.000000,...,0.002070,0.000000,0.001035,0.001035,0.000000,0.004141,0.004141,0.000000,0.003106,0.006211
3,Northeast,0.004202,0.000000,0.000000,0.000000,0.000000,0.037815,0.00000,0.016807,0.000000,...,0.000000,0.000000,0.000000,0.004202,0.000000,0.000000,0.004202,0.000000,0.008403,0.000000
4,Northwest,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021053,0.000000,0.000000
5,South,0.000000,0.000000,0.000000,0.000000,0.000000,0.019943,0.00000,0.000000,0.000000,...,0.000000,0.005698,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008547
6,Southeast,0.001224,0.000000,0.000000,0.000000,0.001224,0.035496,0.00000,0.000000,0.004896,...,0.002448,0.000000,0.000000,0.000000,0.004896,0.002448,0.003672,0.000000,0.008568,0.008568
7,Southwest,0.000000,0.000000,0.003226,0.003226,0.000000,0.035484,0.00000,0.000000,0.000000,...,0.006452,0.000000,0.003226,0.000000,0.000000,0.003226,0.000000,0.012903,0.000000,0.000000
8,West,0.011236,0.000000,0.005618,0.005618,0.000000,0.056180,0.00000,0.000000,0.000000,...,0.011236,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016854,0.000000,0.000000


Grouping the neighborhoods identified 9 unique values (the original geological locations), decreasing the dataframe to 9 rows and it also automatically sorted the neighborhoods

Let's identify the top 10 venues in each geological location.

In [32]:
num_top_venues = 10

for hood in baltimore_grouped['Location']:
    print("----"+hood+"----")
    temp = baltimore_grouped[baltimore_grouped['Location'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central----
                 venue  freq
0       Sandwich Place  0.05
1                Hotel  0.04
2          Coffee Shop  0.04
3    Convenience Store  0.03
4  American Restaurant  0.03
5              Theater  0.03
6               Lounge  0.02
7                  Bar  0.02
8    Indian Restaurant  0.02
9                 Café  0.02


----East----
                  venue  freq
0   American Restaurant  0.05
1     Convenience Store  0.04
2           Coffee Shop  0.04
3              Bus Stop  0.03
4               Theater  0.03
5  Fast Food Restaurant  0.03
6    Chinese Restaurant  0.03
7        Sandwich Place  0.03
8            Shoe Store  0.02
9                  Park  0.02


----North----
                 venue  freq
0          Pizza Place  0.04
1          Coffee Shop  0.04
2                  Bar  0.04
3  American Restaurant  0.03
4       Sandwich Place  0.03
5             Pharmacy  0.03
6         Liquor Store  0.02
7   Chinese Restaurant  0.02
8             Bus Stop  0.02
9    Convenien

#### Adding the data to a pandas dataframe

In [33]:
# Function sorts the values in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
# Identifies the top 10 venues for each neighborhood and outputs them in a df
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Location']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Location'] = baltimore_grouped['Location']

for ind in np.arange(baltimore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(baltimore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central,Sandwich Place,Hotel,Coffee Shop,Convenience Store,Theater,American Restaurant,Café,Bar,Pizza Place,Indian Restaurant
1,East,American Restaurant,Convenience Store,Coffee Shop,Theater,Bus Stop,Sandwich Place,Fast Food Restaurant,Chinese Restaurant,Intersection,Grocery Store
2,North,Coffee Shop,Pizza Place,Bar,American Restaurant,Pharmacy,Sandwich Place,Café,Convenience Store,Chinese Restaurant,Liquor Store
3,Northeast,Italian Restaurant,American Restaurant,Convenience Store,Seafood Restaurant,Chinese Restaurant,Fried Chicken Joint,Pizza Place,Bar,Discount Store,Bus Stop
4,Northwest,Park,Liquor Store,American Restaurant,Chinese Restaurant,Arts & Crafts Store,Grocery Store,Pharmacy,Discount Store,Market,Convenience Store
5,South,Bar,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Park,Mexican Restaurant,Seafood Restaurant,Gym,Liquor Store
6,Southeast,Bar,Italian Restaurant,American Restaurant,Mexican Restaurant,Seafood Restaurant,Pizza Place,Hotel,Coffee Shop,Greek Restaurant,Clothing Store
7,Southwest,Sandwich Place,American Restaurant,Convenience Store,Discount Store,History Museum,Pizza Place,Fried Chicken Joint,Coffee Shop,Hotel,Pub
8,West,American Restaurant,Fried Chicken Joint,Sandwich Place,Pizza Place,Convenience Store,Bus Stop,Bank,Chinese Restaurant,Park,Grocery Store


## Clustering the neighborhoods

Import the necessary packages for this section

In [35]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [36]:
# set number of clusters
kclusters = 5

baltimore_grouped_clustering = baltimore_grouped.drop('Location', 1)
baltimore_grouped_clustering.head(10)

,ATM,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,...,Vietnamese Restaurant,Volleyball Court,Waterfall,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.000000,0.000000,0.008000,0.002667,0.000000,0.029333,0.00000,0.010667,0.000000,0.000000,...,0.008000,0.000000,0.000000,0.000000,0.000000,0.008000,0.000000,0.000000,0.000000,0.004000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.051724,0.00000,0.000000,0.000000,0.011494,...,0.005747,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.005747
2,0.005176,0.001035,0.001035,0.000000,0.000000,0.033126,0.00207,0.000000,0.000000,0.009317,...,0.002070,0.000000,0.001035,0.001035,0.000000,0.004141,0.004141,0.000000,0.003106,0.006211
3,0.004202,0.000000,0.000000,0.000000,0.000000,0.037815,0.00000,0.016807,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.004202,0.000000,0.000000,0.004202,0.000000,0.008403,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021053,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.019943,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.005698,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008547
6,0.001224,0.000000,0.000000,0.000000,0.001224,0.035496,0.00000,0.000000,0.004896,0.000000,...,0.002448,0.000000,0.000000,0.000000,0.004896,0.002448,0.003672,0.000000,0.008568,0.008568
7,0.000000,0.000000,0.003226,0.003226,0.000000,0.035484,0.00000,0.000000,0.000000,0.000000,...,0.006452,0.000000,0.003226,0.000000,0.000000,0.003226,0.000000,0.012903,0.000000,0.000000
8,0.011236,0.000000,0.005618,0.005618,0.000000,0.056180,0.00000,0.000000,0.000000,0.000000,...,0.011236,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016854,0.000000,0.000000


Here we are running the k-means clustering and using a random state so the results can be replicated

In [37]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(baltimore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 0, 3, 1, 2, 3, 0, 4], dtype=int32)

We are going to merge the new dataframe with the one we just created in the previous section. The merged dataframe should include Postal Code, Borough, Neighborhood, Lat, Long, the Clustering label just identified, and the top 7 venues for each neighborhood.

In [38]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [39]:
baltimore_merged = df_final

# merge baltimore_grouped with df_final to add latitude/longitude for each neighborhood
baltimore_merged = baltimore_merged.join(neighborhoods_venues_sorted.set_index('Location'), on='Location')

In [40]:
baltimore_merged.head()

,Neighborhood,Location,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abell,North,39.326139,-76.611737,0,Coffee Shop,Pizza Place,Bar,American Restaurant,Pharmacy,Sandwich Place,Café,Convenience Store,Chinese Restaurant,Liquor Store
1,Allendale,West,39.291687,-76.676942,4,American Restaurant,Fried Chicken Joint,Sandwich Place,Pizza Place,Convenience Store,Bus Stop,Bank,Chinese Restaurant,Park,Grocery Store
2,Arcadia,Northeast,39.334056,-76.566500,3,Italian Restaurant,American Restaurant,Convenience Store,Seafood Restaurant,Chinese Restaurant,Fried Chicken Joint,Pizza Place,Bar,Discount Store,Bus Stop
3,Arlington,Northwest,39.348692,-76.682666,1,Park,Liquor Store,American Restaurant,Chinese Restaurant,Arts & Crafts Store,Grocery Store,Pharmacy,Discount Store,Market,Convenience Store
4,Armistead Gardens,Northeast,39.306646,-76.556279,3,Italian Restaurant,American Restaurant,Convenience Store,Seafood Restaurant,Chinese Restaurant,Fried Chicken Joint,Pizza Place,Bar,Discount Store,Bus Stop


We will now put all the values on a map of Baltimore

In [41]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(baltimore_merged['Latitude'], baltimore_merged['Longitude'], baltimore_merged['Neighborhood'], baltimore_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining the clusters created

First cluster: Largest Cluster- Most Diverse

In [42]:
baltimore_merged.loc[baltimore_merged['Cluster Labels'] == 0, baltimore_merged.columns[[0] + list(range(4, baltimore_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abell,0,Coffee Shop,Pizza Place,Bar,American Restaurant,Pharmacy,Sandwich Place,Café,Convenience Store,Chinese Restaurant,Liquor Store
9,Barre Circle,0,Sandwich Place,American Restaurant,Convenience Store,Discount Store,History Museum,Pizza Place,Fried Chicken Joint,Coffee Shop,Hotel,Pub
11,Beechfield,0,Sandwich Place,American Restaurant,Convenience Store,Discount Store,History Museum,Pizza Place,Fried Chicken Joint,Coffee Shop,Hotel,Pub
17,Bolton Hill,0,Sandwich Place,Hotel,Coffee Shop,Convenience Store,Theater,American Restaurant,Café,Bar,Pizza Place,Indian Restaurant
24,Cameron Village,0,Coffee Shop,Pizza Place,Bar,American Restaurant,Pharmacy,Sandwich Place,Café,Convenience Store,Chinese Restaurant,Liquor Store
...,...,...,...,...,...,...,...,...,...,...,...,...
213,Woodbourne Heights,0,Coffee Shop,Pizza Place,Bar,American Restaurant,Pharmacy,Sandwich Place,Café,Convenience Store,Chinese Restaurant,Liquor Store
214,Woodbourne-McCabe,0,Coffee Shop,Pizza Place,Bar,American Restaurant,Pharmacy,Sandwich Place,Café,Convenience Store,Chinese Restaurant,Liquor Store
217,Wyman Park,0,Coffee Shop,Pizza Place,Bar,American Restaurant,Pharmacy,Sandwich Place,Café,Convenience Store,Chinese Restaurant,Liquor Store
218,Wyndhurst,0,Coffee Shop,Pizza Place,Bar,American Restaurant,Pharmacy,Sandwich Place,Café,Convenience Store,Chinese Restaurant,Liquor Store


Second cluster: Park/Drinking

In [43]:
baltimore_merged.loc[baltimore_merged['Cluster Labels'] == 1, baltimore_merged.columns[[0] + list(range(4, baltimore_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Arlington,1,Park,Liquor Store,American Restaurant,Chinese Restaurant,Arts & Crafts Store,Grocery Store,Pharmacy,Discount Store,Market,Convenience Store
6,Ashburton,1,Park,Liquor Store,American Restaurant,Chinese Restaurant,Arts & Crafts Store,Grocery Store,Pharmacy,Discount Store,Market,Convenience Store
23,Callaway-Garrison,1,Park,Liquor Store,American Restaurant,Chinese Restaurant,Arts & Crafts Store,Grocery Store,Pharmacy,Discount Store,Market,Convenience Store
31,Central Park Heights,1,Park,Liquor Store,American Restaurant,Chinese Restaurant,Arts & Crafts Store,Grocery Store,Pharmacy,Discount Store,Market,Convenience Store
36,Cheswolde,1,Park,Liquor Store,American Restaurant,Chinese Restaurant,Arts & Crafts Store,Grocery Store,Pharmacy,Discount Store,Market,Convenience Store
40,Cross Country,1,Park,Liquor Store,American Restaurant,Chinese Restaurant,Arts & Crafts Store,Grocery Store,Pharmacy,Discount Store,Market,Convenience Store
45,Dolfield,1,Park,Liquor Store,American Restaurant,Chinese Restaurant,Arts & Crafts Store,Grocery Store,Pharmacy,Discount Store,Market,Convenience Store
46,Dorchester,1,Park,Liquor Store,American Restaurant,Chinese Restaurant,Arts & Crafts Store,Grocery Store,Pharmacy,Discount Store,Market,Convenience Store
49,East Arlington,1,Park,Liquor Store,American Restaurant,Chinese Restaurant,Arts & Crafts Store,Grocery Store,Pharmacy,Discount Store,Market,Convenience Store
59,Fallstaff,1,Park,Liquor Store,American Restaurant,Chinese Restaurant,Arts & Crafts Store,Grocery Store,Pharmacy,Discount Store,Market,Convenience Store


Third cluster: Social groups

In [44]:
baltimore_merged.loc[baltimore_merged['Cluster Labels'] == 2, baltimore_merged.columns[[0] + list(range(4, baltimore_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Arundel Cove,2,Bar,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Park,Mexican Restaurant,Seafood Restaurant,Gym,Liquor Store
20,Brooklyn,2,Bar,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Park,Mexican Restaurant,Seafood Restaurant,Gym,Liquor Store
21,Brooklyn Homes,2,Bar,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Park,Mexican Restaurant,Seafood Restaurant,Gym,Liquor Store
35,Cherry Hill,2,Bar,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Park,Mexican Restaurant,Seafood Restaurant,Gym,Liquor Store
41,Curtis Bay,2,Bar,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Park,Mexican Restaurant,Seafood Restaurant,Gym,Liquor Store
60,Federal Hill,2,Bar,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Park,Mexican Restaurant,Seafood Restaurant,Gym,Liquor Store
84,Harborview,2,Bar,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Park,Mexican Restaurant,Seafood Restaurant,Gym,Liquor Store
87,Hawkins Point,2,Bar,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Park,Mexican Restaurant,Seafood Restaurant,Gym,Liquor Store
109,Lakeland,2,Bar,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Park,Mexican Restaurant,Seafood Restaurant,Gym,Liquor Store
117,Locust Point,2,Bar,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Park,Mexican Restaurant,Seafood Restaurant,Gym,Liquor Store


Fourth Cluster: Restaurants

In [45]:
baltimore_merged.loc[baltimore_merged['Cluster Labels'] == 3, baltimore_merged.columns[[0] + list(range(4, baltimore_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Arcadia,3,Italian Restaurant,American Restaurant,Convenience Store,Seafood Restaurant,Chinese Restaurant,Fried Chicken Joint,Pizza Place,Bar,Discount Store,Bus Stop
4,Armistead Gardens,3,Italian Restaurant,American Restaurant,Convenience Store,Seafood Restaurant,Chinese Restaurant,Fried Chicken Joint,Pizza Place,Bar,Discount Store,Bus Stop
7,Baltimore Highlands,3,Bar,Italian Restaurant,American Restaurant,Mexican Restaurant,Seafood Restaurant,Pizza Place,Hotel,Coffee Shop,Greek Restaurant,Clothing Store
10,Bayview,3,Bar,Italian Restaurant,American Restaurant,Mexican Restaurant,Seafood Restaurant,Pizza Place,Hotel,Coffee Shop,Greek Restaurant,Clothing Store
12,Belair-Edison,3,Italian Restaurant,American Restaurant,Convenience Store,Seafood Restaurant,Chinese Restaurant,Fried Chicken Joint,Pizza Place,Bar,Discount Store,Bus Stop
13,Belair-Parkside,3,Italian Restaurant,American Restaurant,Convenience Store,Seafood Restaurant,Chinese Restaurant,Fried Chicken Joint,Pizza Place,Bar,Discount Store,Bus Stop
15,Beverly Hills,3,Italian Restaurant,American Restaurant,Convenience Store,Seafood Restaurant,Chinese Restaurant,Fried Chicken Joint,Pizza Place,Bar,Discount Store,Bus Stop
18,Brewer's Hill,3,Bar,Italian Restaurant,American Restaurant,Mexican Restaurant,Seafood Restaurant,Pizza Place,Hotel,Coffee Shop,Greek Restaurant,Clothing Store
22,Butchers Hill,3,Bar,Italian Restaurant,American Restaurant,Mexican Restaurant,Seafood Restaurant,Pizza Place,Hotel,Coffee Shop,Greek Restaurant,Clothing Store
25,Canton,3,Bar,Italian Restaurant,American Restaurant,Mexican Restaurant,Seafood Restaurant,Pizza Place,Hotel,Coffee Shop,Greek Restaurant,Clothing Store


Fifth Cluster: Quick bites 

In [46]:
baltimore_merged.loc[baltimore_merged['Cluster Labels'] == 4, baltimore_merged.columns[[0] + list(range(4, baltimore_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Allendale,4,American Restaurant,Fried Chicken Joint,Sandwich Place,Pizza Place,Convenience Store,Bus Stop,Bank,Chinese Restaurant,Park,Grocery Store
8,Barclay,4,American Restaurant,Convenience Store,Coffee Shop,Theater,Bus Stop,Sandwich Place,Fast Food Restaurant,Chinese Restaurant,Intersection,Grocery Store
14,Berea,4,American Restaurant,Convenience Store,Coffee Shop,Theater,Bus Stop,Sandwich Place,Fast Food Restaurant,Chinese Restaurant,Intersection,Grocery Store
16,Biddle Street,4,American Restaurant,Convenience Store,Coffee Shop,Theater,Bus Stop,Sandwich Place,Fast Food Restaurant,Chinese Restaurant,Intersection,Grocery Store
19,Broadway East,4,American Restaurant,Convenience Store,Coffee Shop,Theater,Bus Stop,Sandwich Place,Fast Food Restaurant,Chinese Restaurant,Intersection,Grocery Store
39,Coppin Heights,4,American Restaurant,Fried Chicken Joint,Sandwich Place,Pizza Place,Convenience Store,Bus Stop,Bank,Chinese Restaurant,Park,Grocery Store
43,Darley Park,4,American Restaurant,Convenience Store,Coffee Shop,Theater,Bus Stop,Sandwich Place,Fast Food Restaurant,Chinese Restaurant,Intersection,Grocery Store
48,Eager Park,4,American Restaurant,Convenience Store,Coffee Shop,Theater,Bus Stop,Sandwich Place,Fast Food Restaurant,Chinese Restaurant,Intersection,Grocery Store
50,East Baltimore Midway,4,American Restaurant,Convenience Store,Coffee Shop,Theater,Bus Stop,Sandwich Place,Fast Food Restaurant,Chinese Restaurant,Intersection,Grocery Store
51,Easterwood,4,American Restaurant,Fried Chicken Joint,Sandwich Place,Pizza Place,Convenience Store,Bus Stop,Bank,Chinese Restaurant,Park,Grocery Store


### Examination by geological location
After using k-means on the geological location, the clusters were grouped similar to the location. We will now look complete the same analysis, but this time we will be using the neighborhood names instead to see if the grouping is similar or different.

### Start of Neighborhood Analysis

In [47]:
def getNearbyVenues2(names, latitudes, longitudes, radius=500):
    
    venues_list2=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list2.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues2 = pd.DataFrame([item for venue_list2 in venues_list2 for item in venue_list2])
    nearby_venues2.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues2)

In [48]:
baltimore2_venues = getNearbyVenues2(names=df_final['Neighborhood'],
                                   latitudes=df_final['Latitude'],
                                   longitudes=df_final['Longitude']
                                  )

Abell
Allendale
Arcadia
Arlington
Armistead Gardens
Arundel Cove
Ashburton
Baltimore Highlands
Barclay
Barre Circle
Bayview
Beechfield
Belair-Edison
Belair-Parkside
Berea
Beverly Hills
Biddle Street
Bolton Hill
Brewer's Hill
Broadway East
Brooklyn
Brooklyn Homes
Butchers Hill
Callaway-Garrison
Cameron Village
Canton
Carrollton Ridge
Cathedral Hill
Cedarcroft
Cedmont
Cedonia
Central Park Heights
Charles Center
Charles North
Charles Village
Cherry Hill
Cheswolde
Christopher
Cold Spring
Coppin Heights
Cross Country
Curtis Bay
Cylburn
Darley Park
Dickeyville
Dolfield
Dorchester
Druid Heights
Eager Park
East Arlington
East Baltimore Midway
Easterwood
Eastwood
Edmondson
Edmondson Village
Ednor Gardens-Lakeside
Ellwood Park
Evergreen
Fairmount
Fallstaff
Federal Hill
Fells Point
Fells Prospect
Forest Park
Frankford
Franklin Square
Franklintown
Gardenville
Gay Street
Glen
Glen Oaks
Glenham-Belford
Govans
Graceland Park
Greektown
Greenmount West
Greenspring
Grove Park
Guilford
Gwynn Oak
Hampden


In [49]:
print(baltimore2_venues.shape)
baltimore2_venues.head()

(3879, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abell,39.326139,-76.611737,32nd Street Farmer's Market,39.327362,-76.610851,Farmers Market
1,Abell,39.326139,-76.611737,Pete's Grille,39.327064,-76.609593,Breakfast Spot
2,Abell,39.326139,-76.611737,The Book Thing,39.325253,-76.610272,Bookstore
3,Abell,39.326139,-76.611737,Normal's Books & Records,39.326012,-76.609903,Record Shop
4,Abell,39.326139,-76.611737,Peabody Heights Brewery,39.324744,-76.610819,Brewery


In [50]:
print(baltimore2_venues.shape)
baltimore2_venues.head()

(3879, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abell,39.326139,-76.611737,32nd Street Farmer's Market,39.327362,-76.610851,Farmers Market
1,Abell,39.326139,-76.611737,Pete's Grille,39.327064,-76.609593,Breakfast Spot
2,Abell,39.326139,-76.611737,The Book Thing,39.325253,-76.610272,Bookstore
3,Abell,39.326139,-76.611737,Normal's Books & Records,39.326012,-76.609903,Record Shop
4,Abell,39.326139,-76.611737,Peabody Heights Brewery,39.324744,-76.610819,Brewery


In [51]:
# one hot encoding
baltimore2_onehot = pd.get_dummies(baltimore2_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
baltimore2_onehot['Neighborhood'] = baltimore2_venues['Neighborhood']

In [52]:
# move neighborhood column to the first column
neighborhood = baltimore2_onehot['Neighborhood']
baltimore2_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
baltimore2_onehot.insert(0, 'Neighborhood', neighborhood)
baltimore2_onehot

,Neighborhood,ATM,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,...,Vietnamese Restaurant,Volleyball Court,Waterfall,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Abell,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Abell,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Abell,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Abell,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Abell,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3874,York Road,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3875,York Road,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3876,York Road,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3877,York Road,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
baltimore2_onehot.shape

(3879, 290)

In [54]:
baltimore2_grouped = baltimore2_onehot.groupby('Neighborhood').mean().reset_index()
baltimore2_grouped

,Neighborhood,ATM,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,...,Vietnamese Restaurant,Volleyball Court,Waterfall,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Abell,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
1,Allendale,0.0,0.0,0.0,0.0,0.0,0.333333,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
2,Arcadia,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
3,Arlington,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
4,Armistead Gardens,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,Woodring,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
208,Wyman Park,0.0,0.0,0.0,0.0,0.0,0.000000,0.023256,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.023256,0.0,0.000000,0.023256
209,Wyndhurst,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.090909,0.000000
210,Yale Heights,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000


In [55]:
num_top_venues = 10

for hood in baltimore2_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = baltimore2_grouped[baltimore2_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abell----
                venue  freq
0         Pizza Place  0.07
1  Chinese Restaurant  0.07
2            Pharmacy  0.05
3      Sandwich Place  0.05
4         Coffee Shop  0.04
5       Grocery Store  0.04
6            Bus Stop  0.04
7                 Bar  0.04
8                Café  0.04
9           Bookstore  0.04


----Allendale----
                   venue  freq
0                    Bar  0.33
1    American Restaurant  0.33
2       Basketball Court  0.33
3                    ATM  0.00
4        Organic Grocery  0.00
5     Persian Restaurant  0.00
6  Performing Arts Venue  0.00
7                   Park  0.00
8      Outdoor Sculpture  0.00
9      Other Repair Shop  0.00


----Arcadia----
                   venue  freq
0     Chinese Restaurant   0.5
1      Other Repair Shop   0.5
2                    ATM   0.0
3              Pet Store   0.0
4    Peruvian Restaurant   0.0
5     Persian Restaurant   0.0
6  Performing Arts Venue   0.0
7                   Park   0.0
8      Outdoor Sculp

In [56]:
# Function sorts the values in descending order
def return_most_common_venues2(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [70]:
# Identifies the top 10 venues for each neighborhood and outputs them in a df
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted3 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted3['Neighborhood'] = baltimore2_grouped['Neighborhood']

for ind in np.arange(baltimore2_grouped.shape[0]):
    neighborhoods_venues_sorted3.iloc[ind, 1:] = return_most_common_venues2(baltimore2_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted3

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abell,Pizza Place,Chinese Restaurant,Sandwich Place,Pharmacy,Bus Stop,Coffee Shop,Café,Grocery Store,Convenience Store,Bookstore
1,Allendale,Bar,American Restaurant,Basketball Court,Yoga Studio,Field,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
2,Arcadia,Chinese Restaurant,Other Repair Shop,Yoga Studio,Electronics Store,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
3,Arlington,Caribbean Restaurant,Discount Store,Southern / Soul Food Restaurant,Thrift / Vintage Store,Check Cashing Service,Chinese Restaurant,Pharmacy,Convenience Store,Department Store,Design Studio
4,Armistead Gardens,Convenience Store,Rental Service,Sandwich Place,Bar,Seafood Restaurant,Fish Market,Field,Flea Market,Fast Food Restaurant,Farmers Market
...,...,...,...,...,...,...,...,...,...,...,...
207,Woodring,Baseball Field,German Restaurant,Yoga Studio,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
208,Wyman Park,Bakery,Pizza Place,Pet Store,Indian Restaurant,Café,Coffee Shop,Yoga Studio,Bookstore,Shopping Mall,Shoe Store
209,Wyndhurst,Bank,Spa,Coffee Shop,Flower Shop,Furniture / Home Store,Toy / Game Store,Pharmacy,Clothing Store,Grocery Store,Women's Store
210,Yale Heights,Home Service,Yoga Studio,Dumpling Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market


# Clustering Baltimore Neighborhoods

In [71]:
# set number of clusters
kclusters = 5

baltimore2_grouped_clustering = baltimore2_grouped.drop('Neighborhood', 1)
baltimore2_grouped_clustering.head(10)

,ATM,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,...,Vietnamese Restaurant,Volleyball Court,Waterfall,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.00,0.333333,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.25,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.00,0.150000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.00,0.062500,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(baltimore2_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 0, 0, 2, 0, 2, 2], dtype=int32)

In [73]:
# add clustering labels
neighborhoods_venues_sorted3.insert(0, 'Cluster Labels', kmeans.labels_)

In [74]:
baltimore2_merged = df_final

# merge baltimore_grouped with df_final to add latitude/longitude for each neighborhood
baltimore2_merged = baltimore2_merged.join(neighborhoods_venues_sorted3.set_index('Neighborhood'), on='Neighborhood')

baltimore2_merged.head()


,Neighborhood,Location,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abell,North,39.326139,-76.611737,2.0,Pizza Place,Chinese Restaurant,Sandwich Place,Pharmacy,Bus Stop,Coffee Shop,Café,Grocery Store,Convenience Store,Bookstore
1,Allendale,West,39.291687,-76.676942,2.0,Bar,American Restaurant,Basketball Court,Yoga Studio,Field,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
2,Arcadia,Northeast,39.334056,-76.566500,2.0,Chinese Restaurant,Other Repair Shop,Yoga Studio,Electronics Store,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
3,Arlington,Northwest,39.348692,-76.682666,2.0,Caribbean Restaurant,Discount Store,Southern / Soul Food Restaurant,Thrift / Vintage Store,Check Cashing Service,Chinese Restaurant,Pharmacy,Convenience Store,Department Store,Design Studio
4,Armistead Gardens,Northeast,39.306646,-76.556279,0.0,Convenience Store,Rental Service,Sandwich Place,Bar,Seafood Restaurant,Fish Market,Field,Flea Market,Fast Food Restaurant,Farmers Market


In [75]:
baltimore2_merged.loc[baltimore2_merged['Cluster Labels'] == 0, baltimore2_merged.columns[[0] + list(range(4, baltimore2_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Armistead Gardens,0.0,Convenience Store,Rental Service,Sandwich Place,Bar,Seafood Restaurant,Fish Market,Field,Flea Market,Fast Food Restaurant,Farmers Market
5,Arundel Cove,0.0,Gym,Harbor / Marina,Convenience Store,Baseball Field,Food Court,Dance Studio,Event Service,Creperie,Food & Drink Shop,Food
7,Baltimore Highlands,0.0,Convenience Store,Bar,Airport,Light Rail Station,Farmers Market,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm
13,Belair-Parkside,0.0,Convenience Store,River,Chinese Restaurant,Lounge,Flower Shop,Flea Market,Fish Market,Field,Food,Fast Food Restaurant
14,Berea,0.0,Wings Joint,Bus Stop,Market,Convenience Store,Food,Farmers Market,Event Service,Event Space,Exhibit,Falafel Restaurant
35,Cherry Hill,0.0,Dessert Shop,Convenience Store,Dog Run,Auto Garage,Light Rail Station,Dive Bar,Fast Food Restaurant,Exhibit,Falafel Restaurant,Farm
45,Dolfield,0.0,Convenience Store,Grocery Store,Arts & Crafts Store,Farmers Market,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Fast Food Restaurant
64,Frankford,0.0,Convenience Store,Indian Restaurant,Shopping Mall,Chinese Restaurant,Farmers Market,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm
71,Glenham-Belford,0.0,Convenience Store,Intersection,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
90,Hillen,0.0,Sculpture Garden,Bus Stop,Monument / Landmark,Convenience Store,Cupcake Shop,Event Service,Food & Drink Shop,Food,Flower Shop,Flea Market


In [76]:
baltimore2_merged.loc[baltimore2_merged['Cluster Labels'] == 1, baltimore2_merged.columns[[0] + list(range(4, baltimore2_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Broadway East,1.0,Park,Bar,Breakfast Spot,Fast Food Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Yoga Studio
30,Cedonia,1.0,Park,Bar,Deli / Bodega,Yoga Studio,Fast Food Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
31,Central Park Heights,1.0,Park,Spa,Market,Farmers Market,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Yoga Studio
49,East Arlington,1.0,Liquor Store,Auto Workshop,Park,Farm,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farmers Market
73,Graceland Park,1.0,Sports Bar,Bar,Yoga Studio,Fast Food Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Field
79,Gwynn Oak,1.0,Park,Business Service,Yoga Studio,Food Stand,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
85,Harlem Park,1.0,Park,Bar,American Restaurant,Bus Stop,Playground,Farmers Market,Event Service,Event Space,Exhibit,Falafel Restaurant
97,Idlewood,1.0,Park,Yoga Studio,Convenience Store,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
119,Lucille Park,1.0,Park,Liquor Store,Deli / Bodega,Arts & Crafts Store,Pizza Place,Market,Flower Shop,Flea Market,Fish Market,Field
154,Pen Lucy,1.0,Bar,Yoga Studio,Food Truck,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


In [77]:
baltimore2_merged.loc[baltimore2_merged['Cluster Labels'] == 2, baltimore2_merged.columns[[0] + list(range(4, baltimore2_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abell,2.0,Pizza Place,Chinese Restaurant,Sandwich Place,Pharmacy,Bus Stop,Coffee Shop,Café,Grocery Store,Convenience Store,Bookstore
1,Allendale,2.0,Bar,American Restaurant,Basketball Court,Yoga Studio,Field,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
2,Arcadia,2.0,Chinese Restaurant,Other Repair Shop,Yoga Studio,Electronics Store,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
3,Arlington,2.0,Caribbean Restaurant,Discount Store,Southern / Soul Food Restaurant,Thrift / Vintage Store,Check Cashing Service,Chinese Restaurant,Pharmacy,Convenience Store,Department Store,Design Studio
6,Ashburton,2.0,Lake,Health & Beauty Service,Arts & Crafts Store,Electronics Store,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
...,...,...,...,...,...,...,...,...,...,...,...,...
215,Woodmoor,2.0,Event Service,Business Service,Yoga Studio,Convenience Store,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
216,Woodring,2.0,Baseball Field,German Restaurant,Yoga Studio,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
217,Wyman Park,2.0,Bakery,Pizza Place,Pet Store,Indian Restaurant,Café,Coffee Shop,Yoga Studio,Bookstore,Shopping Mall,Shoe Store
218,Wyndhurst,2.0,Bank,Spa,Coffee Shop,Flower Shop,Furniture / Home Store,Toy / Game Store,Pharmacy,Clothing Store,Grocery Store,Women's Store


In [78]:
baltimore2_merged.loc[baltimore2_merged['Cluster Labels'] == 3, baltimore2_merged.columns[[0] + list(range(4, baltimore2_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Callaway-Garrison,3.0,Grocery Store,Yoga Studio,Food Stand,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
88,Heritage Crossing,3.0,Grocery Store,Yoga Studio,Food Stand,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market


In [79]:
baltimore2_merged.loc[baltimore2_merged['Cluster Labels'] == 4, baltimore2_merged.columns[[0] + list(range(4, baltimore2_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
87,Hawkins Point,4.0,Business Service,Home Service,Yoga Studio,Electronics Store,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
146,Orchard Ridge,4.0,Home Service,Brewery,Yoga Studio,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
219,Yale Heights,4.0,Home Service,Yoga Studio,Dumpling Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
